# Basic visualisation of THAI LFRic simulations

### Synopsis

This notebook goes through the following steps:
1. Load LFRic output using `iris`, thus representing each of the output variables as `iris.cube.Cube`.
2. Regrid the cubes from unstructured grid to regular rectilinear lat-lon grid using `iris-esmf-regrid`.
3. Average the data over time and/or longitude using functions from `aeolus`.
4. Plot the results and save them to the `../plots/` directory.

Special thanks to Paul Earnshaw for the help with regridding LFRic output.

### Import the necessary libraries

Standard library

In [ ]:
import warnings

warnings.filterwarnings("ignore")  # noqa

In [ ]:
from functools import partial

Scientific stack

In [ ]:
import esmf_regrid
import iris
import iris.coord_systems
import iris.fileformats
import iris.plot as iplt
import iris.quickplot as qplt
import matplotlib.colors as mcol
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
import numpy as np
from esmf_regrid.experimental.unstructured_scheme import (
    MeshToGridESMFRegridder,
    regrid_unstructured_to_rectilinear,
)
from iris.experimental.ugrid import PARSE_UGRID_ON_LOAD

# from iris.experimental import stratify
from matplotlib.offsetbox import AnchoredText

In [ ]:
print(iris.__version__, esmf_regrid.__version__)

In [ ]:
from tqdm.notebook import tqdm

aeolus

In [ ]:
from aeolus.calc import spatial_mean, time_mean, zonal_mean
from aeolus.const import init_const
from aeolus.coord import get_cube_rel_days, get_xy_coords, roll_cube_pm180
from aeolus.io import create_dummy_cube, load_vert_lev
from aeolus.model import um
from aeolus.subset import extract_last_n_days

local scripts

In [ ]:
import mypaths
from lfric_util import clean_attrs  # add_equally_spaced_height_coord
from plot_func import cube_stats_string, figsave, use_style

In [ ]:
def lfric_spatial_mean(cube, model=um):
    cube_copy = cube.copy()
    tmp_coord = iris.coords.AuxCoord(
        points=np.arange(cube.coord(model.x).shape[0]), long_name="mesh_coordinates"
    )

    cube_copy.add_aux_coord(tmp_coord, data_dims=cube.coord_dims(model.x))

    cube_copy.remove_coord(model.x)
    cube_copy.remove_coord(model.y)

    cube_mean = cube_copy.collapsed(tmp_coord.name(), iris.analysis.MEAN)
    cube_mean.remove_coord(tmp_coord)
    return cube_mean


def broadcast_cube(cube, target_cube, **kwargs):
    """Broadcast cube to the shape of another cube with at least 1 matching dimension."""
    kw = {
        "units": cube.units,
        "long_name": cube.long_name,
        "standard_name": cube.standard_name,
        "var_name": cube.var_name,
    }
    kw = {**kw, **kwargs}
    data_bcast = iris.util.broadcast_to_shape(
        cube.data,
        target_cube.shape,
        dim_map=[target_cube.coord_dims(coord)[0] for coord in cube.dim_coords],
    )
    tgt_dim_coords = [(c.copy(), target_cube.coord_dims(c)) for c in target_cube.dim_coords]
    tgt_aux_coords = [(c.copy(), target_cube.coord_dims(c)) for c in target_cube.aux_coords]
    cube_bcast = iris.cube.Cube(
        data=data_bcast,
        dim_coords_and_dims=tgt_dim_coords,
        aux_coords_and_dims=tgt_aux_coords,
        **kw,
    )
    return cube_bcast

### Some global definitions

Apply a custom `matplotlib` style.

In [ ]:
use_style()

Create a path effects object to highlight contours for later

In [ ]:
PE = [PathEffects.withStroke(linewidth=0.5, foreground="w")]

Select the run configuration

In [ ]:
planet = "ben1"
SIM_LABEL = "THAI Ben 1"
const = init_const(planet, directory=mypaths.const)

In [ ]:
kappa = const.dry_air_gas_constant / const.dry_air_spec_heat_press

Common directory where to save plots.

In [ ]:
plot_dir = mypaths.plot / "lfric_plots"

### Load the data into a cube list

Use a callback function to attach a level height coordinate for convenience.

Note: adapt the file path to the one on your machine in the `mypaths.py` script.

In [ ]:
vert_lev = load_vert_lev(mypaths.vert_um / "vertlevs_L38_29t_9s_40km")

In [ ]:
# sim_case = "ben1"
# timestep = 1800
sim_case = "ben2"
timestep = 300
with_restart = False
rundir = f"run_lfric_atm_thai_{sim_case}{'with_restart' if with_restart else '_'}C24_dt-{timestep}p0_intel_64-bit_fast-debug"

In [ ]:
with PARSE_UGRID_ON_LOAD.context():
    exner_ben2 = iris.load_cube(
        str(mypaths.results_lfric / "thai" / sim_case / "1" / rundir / "lfric_initial.nc"), "exner"
    )
    exner_ben1 = iris.load_cube(
        str(
            mypaths.results_lfric
            / "thai"
            / "ben1"
            / "1"
            / "run_lfric_atm_thai_ben1_C24_dt-1800p0_intel_64-bit_fast-debug"
            / "lfric_initial.nc"
        ),
        "exner",
    )
    exner_ben2_changed_cp_rd = iris.load_cube(
        str(
            mypaths.results_lfric
            / "thai"
            / "ben2_changed_cp_rd"
            / "1"
            / "run_lfric_atm_thai_ben2_C24_dt-300p0_intel_64-bit_fast-debug"
            / "lfric_initial.nc"
        ),
        "exner",
    )
    exner_ben2_changed_rd = iris.load_cube(
        str(
            mypaths.results_lfric
            / "thai"
            / "ben2_changed_rd"
            / "1"
            / "run_lfric_atm_thai_ben2_C24_dt-300p0_intel_64-bit_fast-debug"
            / "lfric_initial.nc"
        ),
        "exner",
    )
# for cube in sdump:
#     print(f"{cube.var_name:<40} | {cube.name():<50} | {cube.data.min():<10.1e} | {cube.data.max():<10.1e}")

In [ ]:
qplt.plot(exner_ben1[:, 111], exner_ben1[:, 111].coord("half_levels"), linewidth=5, label="Ben 1: $c_p=1039, r_d=297$")
qplt.plot(exner_ben2[:, 111], exner_ben1[:, 111].coord("half_levels"), label="Ben 2: $c_p=844, r_d=188.92$ (FAILS)")
qplt.plot(exner_ben2_changed_cp_rd[:, 111], exner_ben1[:, 111].coord("half_levels"), label="Ben 2: $c_p=1039, r_d=297$")
qplt.plot(exner_ben2_changed_rd[:, 111], exner_ben1[:, 111].coord("half_levels"), label="Ben 2: $c_p=844, r_d=297$")
plt.legend()

In [ ]:
# with PARSE_UGRID_ON_LOAD.context():
    # sdump = iris.load(str(mypaths.start_dumps_lfric / "exoplanet" / "Quagga" / "thai_ben1_C24L38.nc"))

# cube = sdump.extract_cube("air_density")

# fig, ax = plt.subplots()
# p0 = ax.scatter(
#     cube.coord(um.x).points,
#     cube.coord(um.y).points,
#     c=cube.data[0, ...],
#     s=2**4, 
# )
# fig.colorbar(p0)
# print(f"{'var_name':<40} | {'min':<10} | {'max':<10}")
# print("-"*63)
# for cube in sdump:
#     print(f"{cube.var_name:<40} | {cube.data.min():<10.1e} | {cube.data.max():<10.1e}")

In [ ]:
fnames = [
    str(i)
    for i in sorted(
        (mypaths.results_lfric / "thai" / sim_case).glob(f"*/{rundir}/lfric_diag.nc"),
        key=lambda x: int(x.parent.parent.name),
    )
]
fnames = fnames[10:]

In [ ]:
%%time
with PARSE_UGRID_ON_LOAD.context():
    # cl = iris.load(fnames, callback=clean_attrs)
    cl_no_callback = iris.load(fnames)

In [ ]:
%%time
# cl = iris.cube.CubeList()
for cube in cl_no_callback:
    try:
        cube.attributes.pop("timeStamp")
        cube.attributes.pop("uuid")
    except KeyError:
        pass
cl = cl_no_callback.concatenate(check_aux_coords=False)

Print available names for key variables.

In [ ]:
for cube in cl.extract(
    [
        "surface_tile_temperature",
        "air_potential_temperature",
        "eastward_wind",
        "northward_wind",
        "vertical wind on physics points",
    ]
):
    print(f"{cube.long_name:<50} | {cube.var_name:<20}")

In [ ]:
# sfc_tile_frac = cl.extract_cube("surface_tile_fraction")
sfc_tile_temp = cl.extract_cube("surface_tile_temperature")[:, 9, :]

In [ ]:
plt.plot(get_cube_rel_days(sfc_tile_temp), lfric_spatial_mean(sfc_tile_temp).data, marker=".")

In [ ]:
u = cl.extract_cube(iris.Constraint(cube_func=lambda cube: cube.var_name == "u1"))
v = cl.extract_cube(iris.Constraint(cube_func=lambda cube: cube.var_name == "u2"))

In [ ]:
xy_slice = slice(None, None, 2)

In [ ]:
fig, ax = plt.subplots()
p0 = ax.scatter(
    sfc_tile_temp.coord(um.x).points,
    sfc_tile_temp.coord(um.y).points,
    c=sfc_tile_temp.data[-1, ...],
)
# fig.colorbar(p0)

ax.quiver(
    u.coord(um.x).points[xy_slice],
    u.coord(um.y).points[xy_slice],
    u.data[-1, 20, xy_slice],
    v.data[-1, 20, xy_slice],
)

In [ ]:
theta = cl.extract_cube(iris.Constraint(cube_func=lambda cube: cube.var_name == "theta"))

theta_ss = theta.extract(iris.Constraint(full_levels=lambda x: x < 6, latitude=0, longitude=0))[
    ..., 0
]

fig, ax = plt.subplots()
ax.pcolormesh(theta_ss.coord("time").points, theta_ss.coord("full_levels").points, theta_ss.data.T)

### Regrid the data to rectilinear latitude-longitude grid

Extract the `eastward_wind` in W3 points (otherwise regridding fails for some reason).

In [ ]:
u_w3 = cl.extract_cube(iris.Constraint(cube_func=lambda cube: cube.var_name == "u1_in_w3"))

In [ ]:
exner = cl.extract_cube(iris.Constraint(cube_func=lambda cube: cube.var_name == "exner"))

$$\Pi=\left(\frac{p}{p_{0}}\right)^{R_{d} / c_{p}}=\frac{T}{\theta}$$

In [ ]:
pres = exner.copy(data=const.reference_surface_pressure.data * (exner.data ** (1 / kappa.data)))
pres.rename("air_pressure")
pres.units = "Pa"

In [ ]:
pres_tm = time_mean(extract_last_n_days(pres, 61))

u_w3_tm = time_mean(extract_last_n_days(u_w3, 61))

Create a dummy 2D cube with a UM-like grid. Use +/- 180 degrees to match LFRic, for convenience

In [ ]:
tgt_cube = create_dummy_cube(
    nlat=90, nlon=144, pm180=True
)  # Use a non-standard number of points to match UoE SA suites

In [ ]:
u_reg_tm = regrid_unstructured_to_rectilinear(u_w3_tm, tgt_cube)
pres_reg_tm = regrid_unstructured_to_rectilinear(pres_tm, tgt_cube)

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.set_ylim(1000, 1)
ax.set_yticks([1000, 100, 10, 1])
ax.set_yticklabels([1000, 100, 10, 1])
p0 = ax.contourf(
    u_reg_tm.coord("latitude").points,
    spatial_mean(pres_reg_tm).data * 1e-2,
    zonal_mean(u_reg_tm).data,
    cmap="RdYlBu_r",
    levels=np.arange(-100, 101, 10),
    extend="both",
)
p1 = ax.contour(
    u_reg_tm.coord("latitude").points,
    spatial_mean(pres_reg_tm).data * 1e-2,
    zonal_mean(u_reg_tm).data,
    colors="#333333",
    levels=np.arange(-100, 101, 10),
)
ax.clabel(p1, fmt="%.0f", colors="#333333")
fig.colorbar(p0)

For regridding many cubes at once, it's better to cache the regridder.

In [ ]:
regridder = MeshToGridESMFRegridder(u, tgt_cube)
cl_reg = iris.cube.CubeList()
for cube in cl:
    try:
        cl_reg.append(regridder(cube))
    except (AssertionError, ValueError):
        # For some cubes this fails
        # TODO: investigate!
        pass